## Exception Handling

>* 예외는 오직 한 가지 일을 하는 함수의 한 부분이어야 합니다

### Bad exception handling case

In [1]:
class DataTransport:
    """An example of an object badly handling exceptions of different levels."""

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("connection error detected: %s", e)
            raise
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

    def connect(self):
        for _ in range(self.retry_n_times):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                    "%s: attempting new connection in %is",
                    e,
                    self.retry_threshold,
                )
                time.sleep(self.retry_threshold)
            else:
                return self.connection
        raise ConnectionError(
            f"Couldn't connect after {self.retry_n_times} times"
        )

    def send(self, data):
        return self.connection.send(data)
    

* 위의 deliver_event()가 다루고 있는 두 개의 예외처리 (ConnectionError, ValueError)는 서로 관련이 없으며 에러 발생시 책임의 소재가 불분명하다
* 따라서 각 예외처리는 세분화된 메소드로 나누어 그 안에서 다뤄지는 것이 옳다

### Better exception handling case

In [2]:
def connect_with_retry(connector, retry_n_times, retry_threshold=5):
    """Tries to establish the connection of <connector> retrying
    <retry_n_times>.

    If it can connect, returns the connection object.
    If it's not possible after the retries, raises ConnectionError

    :param connector:           An object with a `.connect()` method.
    :param retry_n_times int:   The number of times to try to call
                                ``connector.connect()``.
    :param retry_threshold int: The time lapse between retry calls.

    """
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: attempting new connection in %is", e, retry_threshold
            )
            time.sleep(retry_threshold)
    exc = ConnectionError(f"Couldn't connect after {retry_n_times} times")
    logger.exception(exc)
    raise exc


class DataTransport:
    """An example of an object that separates the exception handling by
    abstraction levels.
    """

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        self.connection = connect_with_retry(
            self._connector, self.retry_n_times, self.retry_threshold
        )
        self.send(event)

    def send(self, event):
        try:
            return self.connection.send(event.decode())
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

* 위의 코드에서 ConnectionError는 연결 구현 부분인 connect_with_retry()에서만 다뤄지며, ValueError는 보낼 event를 decode하는 send()에서만 다뤄진다
* 에러 발생시 책임의 소재가 분명하다

## Inheritance

## Keep it Simple (KIS)

## Orthogonality